In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import pybrain as pb
import matplotlib.pyplot as plt

#volume needed
volume = list(range(0,100000, 1))

#This script tells you how many shares you need to purchase to dilute a loss
def dilute_losses(initial_volume, initial_price, current_price):
    ### Inputs and outputs
    #param initial_volume: the initial number of shares purchased
    #param initial_price: the inital price paid per share
    #param current_price: the current stock price
    #output: the number of shares needed to dilute the price and how much that would cost
    ###
    for i in volume:
        a = ((i * current_price) + (initial_volume * initial_price)) / (i + initial_volume)
        if a < (current_price + 0.01):
            print(i, 'new shares at a price of $', current_price, 'need to be purchased to mostly dilute your losses. This will cost roughly $',int(i * current_price))
            break
           

In [ ]:
dilute_losses(100,0.3428,.2389)

In [ ]:
#This script tells you the price per share to obtain a desired value
def new_price(initial_volume, initial_price, desired_value):
    ### Inputs and outputs
    #param initial_volume: the initial number of shares purchased
    #param initial_price: the inital price paid per share
    #param desired_value: the desired value of your holding
    #output: the stock price needed to achieve a given value and how much a stock needs to go up
    ###
    current_value = initial_volume * initial_price
    new_price = desired_value / initial_volume
    print('To obtain a desired value of $', desired_value, 'the price needs to go up to $', new_price, 'a change of $',
         new_price - initial_price)

In [ ]:
new_price(100, 0.5, 100)
new_price(100, 0.5, 200)

In [ ]:
#This script tells you how much of your profit needs to go to taxes, reinvestment, and savings
def distribution_of_profits(profit):
    print('$', profit * 0.4, 'needs to go to taxes')
    print('$', profit * 0.3, 'needs to go to reinvestment')
    print('$', profit * 0.3, 'needs to go to savings')

In [ ]:
distribution_of_profits(100)

In [48]:
#Connecting to yFinance and create a ticker dataframe
def create_ticker_df(ticker_symbols, start_date, end_date, interval = False):
    """" Inputs and outputs
    #param ticker_symbols: a single or list of ticker symbols
    #param start_date: date in YYYY-M-D
    #param end_date: date in YYYY-M-D
    #param interval: if false, no 1 minute resolution data, if true, 1m resolution data
    #output dataframe
    """
    ticker_df = []
    for i in ticker_symbols:
        temp_tick = yf.Ticker(i)
        if interval == False:
            temp_df = temp_tick.history(period='1d', start=start_date, end=end_date)
            temp_df['ticker'] = i
            temp_df['daily_change'] = temp_df['Close'] - temp_df['Open']
            temp_df['change_over_volume'] = temp_df['daily_change'] / temp_df['Volume']
            temp_df['change'] = np.where(temp_df['daily_change'] >= 0, 'increase', 'decrease')
        else:
            temp_df = temp_tick.history(period='1d', start=start_date, end=end_date, interval="1m")
            temp_df['ticker'] = i
            temp_df['minute_change'] = temp_df['Close'] - temp_df['Open']
            temp_df['letter_change'] = np.where(temp_df['minute_change'] > 0.01, 'U', np.where(temp_df['minute_change'] <= -0.01, 'D', 'N'))
        ticker_df.append(temp_df)
    ticker_df = pd.concat(ticker_df, axis=0)
    return(ticker_df)

In [9]:
#Test function
create_ticker_df(['TPIC','F'],'2020-6-8','2020-6-12', interval = True).head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,minute_change,letter_change
Datetime,,,,,,,,,,
2020-06-08 09:30:00-04:00,22.48,23.21,22.48,23.21,12311,0,0,TPIC,0.73,U
2020-06-08 09:31:00-04:00,23.14,23.14,23.14,23.14,513,0,0,TPIC,0.00,N
2020-06-08 09:32:00-04:00,23.16,23.42,23.00,23.42,3175,0,0,TPIC,0.26,U
2020-06-08 09:33:00-04:00,23.50,23.55,23.30,23.50,3892,0,0,TPIC,0.00,N
2020-06-08 09:35:00-04:00,23.53,23.63,23.53,23.56,400,0,0,TPIC,0.03,U


In [10]:
#Create function to build up letter change dataset
def create_letter_change_df(ticker_df, window, split):
    """
    param ticker_df: ticker_df where interval = True
    param window: size of window being investigated
    param split: size of predictor and result strings
    output: dataframe with predictor, results, and ticker information
    """
    lc_df = ticker_df.copy()
    lc_df = lc_df.reset_index()
    lc = list(lc_df['letter_change'])
    out_pred = []
    out_res = []
    for i in range(len(lc)):
        seq = lc[i + 0 : i + window]
        pred = seq[0:split]
        res = seq[split:window]
        out_pred.append(pred)
        out_res.append(res)
    out_df = []
    q = {'predictor': out_pred, 'result': out_res}
    out_df = pd.DataFrame(q)
    out_df['ticker'] = lc_df['ticker'][0]
    out_df['predictor'] = [''.join(map(str, l)) for l in out_df['predictor']]
    out_df['result'] = [''.join(map(str, l)) for l in out_df['result']]
    return out_df

In [11]:
#test the function
ticker_df = create_ticker_df(['XSPA'],'2020-6-8','2020-6-12', interval = True)
test_df = create_letter_change_df(ticker_df, window = 6, split = 3)
test_df.head()

,predictor,result,ticker
0,DUU,UUD,XSPA
1,UUU,UDD,XSPA
2,UUU,DDU,XSPA
3,UUD,DUD,XSPA
4,UDD,UDN,XSPA


In [ ]:
#What are the predictors and results that we see most often?
pairwise_combos = test_df.groupby(['predictor','result']).size().reset_index().rename(columns={0:'count'})
np.quantile(pairwise_combos['count'], [0.1, 0.5, 0.9])
pairwise_common = pairwise_combos.copy()
pairwise_common = pairwise_common[pairwise_common['count'] > np.quantile(pairwise_common['count'], 0.9)]
pairwise_common.sort_values(by=['count'], ascending=False).head()

In [49]:
def test_parameters_for_data_generation(tickers, windows, splits, start_date, end_date):
    """
    This function builds out a dataset of multiple tickers
    param tickers: list of tickers
    param windows: size of windows
    param splits: where the window is split i.e. for a window of 4, split could be 2, where prediction and result length is two
    param start_date: start date
    param end_date: end date
    output: dataframe containing the number of times pairwise combinations of predictions and results occur
    """
    temp_out = []
    global_df = []
    for i in range(len(tickers)):
        temp_ticker = create_ticker_df([tickers[i]],start_date, end_date, interval = True)
        for j in range(len(windows)):
            test_df = create_letter_change_df(temp_ticker, window = windows[j], split = splits[j])
            pairwise_combos = test_df.groupby(['predictor','result']).size().reset_index().rename(columns={0:'count'})
            pairwise_combos = pairwise_combos[pairwise_combos['count'] > np.quantile(pairwise_combos['count'], 0.9)]
            pairwise_combos['ticker'] = tickers[i]
            pairwise_combos['window'] = windows[j]
            temp_out.append(pairwise_combos)
        global_out = pd.concat(temp_out, axis=0, ignore_index=True)
        
            #print(tickers[i], windows[j])
          
    return global_out


In [50]:
#Test function and produce output
output_set = test_parameters_for_data_generation(tickers = ['FET','MVIS'], windows = [4,6,8,10], splits = [2,3,4,5], start_date = '2020-6-8', end_date ='2020-6-12')

#Sort output dataframe
output_set.sort_values(by=['count'], ascending=False)

,predictor,result,count,ticker,window
4,NN,NN,985,FET,4
93,NN,NN,872,MVIS,4
13,NNN,NNN,846,FET,6
34,NNNN,NNNN,751,FET,8
106,NNN,NNN,707,MVIS,6
...,...,...,...,...,...
52,NDNNN,DNNNN,3,FET,10
56,NNDNN,NDNNN,3,FET,10
54,NDNNN,UNNNN,3,FET,10
68,NNNNN,NDNND,3,FET,10


In [ ]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.graph_objects as go

def make_estimates(ticker_symbol, epos, metric = 'Close', look_back = 5, look_forward = 10, start_date = '2019-5-1', end_date = '2020-5-17'):
    temp_df = create_ticker_df([ticker_symbol], start_date, end_date)
    df = temp_df.copy()
    df['Date'] = df.index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_axis(df['Date'], inplace=True)
    df = df[['Date', metric]]
    close_data = df['Close'].values
    close_data = close_data.reshape((-1,1))
    split_percent = 0.70
    split = int(split_percent*len(close_data))
    min_max_scaler = MinMaxScaler()
    close_train =  min_max_scaler.fit_transform(close_data[:split])
    close_test =  min_max_scaler.fit_transform(close_data[split:])
    date_train = df['Date'][:split]
    date_test = df['Date'][split:]
    train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
    test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)
    model = Sequential()
    model.add(
        LSTM(10,
            activation='relu',
            input_shape=(look_back,1))
    )
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    num_epochs = epos
    model.fit_generator(train_generator, epochs=num_epochs, verbose=0)
    prediction = model.predict_generator(test_generator)
    close_train = close_train.reshape((-1))
    close_test = close_test.reshape((-1))
    prediction = model.predict_generator(test_generator)
    close_train = close_train.reshape((-1))
    close_test = close_test.reshape((-1))
    prediction = prediction.reshape((-1))

    trace1 = go.Scatter(
        x = date_train,
        y = close_train,
        mode = 'lines',
        name = 'Data'
    )
    trace2 = go.Scatter(
        x = date_test,
        y = prediction,
        mode = 'lines',
        name = 'Prediction'
    )
    trace3 = go.Scatter(
        x = date_test,
        y = close_test,
        mode='lines',
        name = 'Ground Truth'
    )
    layout = go.Layout(
        title = ticker_symbol,
        xaxis = {'title' : "Date"},
        yaxis = {'title' : "Close"}
    )
    fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
    fig.show()
    close_data = close_data.reshape((-1))

    def predict(num_prediction, model):
        prediction_list = close_data[-look_back:]

        for _ in range(num_prediction):
            x = prediction_list[-look_back:]
            x = x.reshape((1, look_back, 1))
            out = model.predict(x)[0][0]
            prediction_list = np.append(prediction_list, out)
        prediction_list = prediction_list[look_back-1:]

        return prediction_list

    def predict_dates(num_prediction):
        last_date = df['Date'].values[-1]
        prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
        return prediction_dates

    num_prediction = 10
    forecast = predict(num_prediction, model)
    forecast_dates = predict_dates(num_prediction)
    out_frame = []
    q = {'dates':forecast_dates,'price':forecast}
    out_temp = pd.DataFrame(q)
    out_frame.append(out_temp)
    out_frame = pd.concat(out_frame, axis=0)
    out_frame['ticker'] = ticker_symbol
    plt.plot(out_frame['price'])
    plt.xlabel('days from now (5/15/20)')
    plt.ylabel('estimated stock price')
    return(out_frame)


In [ ]:
make_estimates(ticker_symbol = 'FET', epos = 100, start_date = '2020-03-19', end_date = '2020-5-19', look_back = 1)

In [ ]:
ticker_syms = ['MVIS', 'ASM', 'UAVS', 'KTOV', 'IZEA', 'MARK',
               'GNUS', 'IBIO', 'FET', 'AUMN', 'XSPA', 'RNWK', 'SRNE', 'RTTR', 'CBL', 'ZOM',
              'TPIC','TTNP','FSLY','VISL', 'KOS', 'BIOC', 'MARA', 'RIOT', 'INPX', 'OCGN', 
              'CEI', 'EMAN', 'SHIP', 'TAT', 'TOPS', 'RMBL', 'FRSX', 'HTGM', 'DFFN', 'ONTX']

#ticker_syms = ['MVIS', 'KTOV', 'FET', 'MARK', 'TOPS', 'SHIP', 'TAT', 'ZOM', 'CBL']    
out_df = []

for i in ticker_syms:
    blanko = []
    blanko = make_estimates(ticker_symbol = i, epos = 2500, start_date = '2019-5-1', end_date = '2020-5-19')
    out_df.append(blanko)
out_df

In [ ]:
df2 = pd.concat(out_df)

df2.to_csv('Predictions_05_18_20_v2.csv')